# 인기도순 추천
## 인기도의 정의
* 여기에서는 평갓값이 높은 것을 인기도가 높은 영화로 간주한다
* 인기도의 정의는 '클릭 수가 많은 것', '구입이 많은 것', '평갓값이 높은 것' 등 다양하므로, 자사의 서비스에 적합한 정의를 사용한다


In [ ]:
# 부모 폴더의 경로 추가
import sys; sys.path.insert(0, '..')

from util.data_loader import DataLoader
from util.metric_calculator import MetricCalculator

In [ ]:
# Movielens 데이터 로딩
data_loader = DataLoader(num_users=1000, num_test_items=5, data_path='../data/ml-10M100K/')
movielens = data_loader.load()

In [ ]:
import numpy as np

# 평갓값이 높은 영화 확인
movie_stats = movielens.train.groupby(['movie_id', 'title']).agg({'rating': [np.size, np.mean]})
movie_stats.sort_values(by=('rating', 'mean'), ascending=False).head()

In [ ]:
# 임곗값을 도입
movie_stats = movielens.train.groupby(['movie_id', 'title']).agg({'rating': [np.size, np.mean]})
atleast_flg = movie_stats['rating']['size'] >= 100
movies_sorted_by_rating = movie_stats[atleast_flg].sort_values(by=('rating', 'mean'), ascending=False)
movies_sorted_by_rating.head()

In [ ]:
# 인기도 추천
from src.popularity import PopularityRecommender
recommender = PopularityRecommender()
recommend_result = recommender.recommend(movielens, minimum_num_rating=100)

In [ ]:
# 평가
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

In [ ]:
# 임곗값을 변경했을 때의 동작
for minimum_num_rating in [1, 200]:
    recommend_result = recommender.recommend(movielens, minimum_num_rating=minimum_num_rating)
    metrics = metric_calculator.calc(
        movielens.test.rating.tolist(), recommend_result.rating.tolist(),
        movielens.test_user2items, recommend_result.user2items, k=10)
    print(metrics)